# Kinesis Data Analytics for SQL Applications

With Amazon Kinesis Data Analytics for SQL Applications, you can process and analyze streaming data using standard SQL. The service enables you to quickly author and run powerful SQL code against streaming sources to perform time series analytics, feed real-time dashboards, and create real-time metrics.

The service supports ingesting data from Amazon Kinesis Data Streams and Amazon Kinesis Data Firehose streaming sources. Then, you author your SQL code using the interactive editor and test it with live streaming data. You can also configure destinations where you want Kinesis Data Analytics to send the results.

<img src="img/use_case_1_analytics.png" width="80%" align="left">

# Create AWS Lambda Function as Kinesis Data Analytics Destination

Kinesis Data Analytics supports Amazon Kinesis Data Firehose, AWS Lambda, and Amazon Kinesis Data Streams as destinations. Let's create a Lambda function to publish our SQL application results as custom metrics to CloudWatch Metrics.

In [ ]:
import boto3
import sagemaker
import pandas as pd
import json

sess   = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

iam = boto3.Session().client(service_name='iam', region_name=region)
sts = boto3.Session().client(service_name='sts', region_name=region)
account_id = sts.get_caller_identity()['Account']

lam = boto3.Session().client(service_name='lambda', region_name=region)

## Retrieve AWS Lambda Function Name

In [ ]:
%store -r lambda_fn_name

In [ ]:
try:
    lambda_fn_name
except NameError:
    print('+++++++++++++++++++++++++++++++')
    print('[ERROR] Please run all previous notebooks in this section before you continue.')
    print('+++++++++++++++++++++++++++++++')

In [ ]:
print(lambda_fn_name)

## Check IAM Roles Are In Place

In [ ]:
%store -r iam_lambda_role_name

In [ ]:
try:
    iam_lambda_role_name
except NameError:
    print('+++++++++++++++++++++++++++++++')
    print('[ERROR] Please run all previous notebooks in this section before you continue.')
    print('+++++++++++++++++++++++++++++++')

In [ ]:
print(iam_lambda_role_name)

In [ ]:
%store -r iam_lambda_role_passed

In [ ]:
try:
    iam_lambda_role_passed
except NameError:
    print('+++++++++++++++++++++++++++++++')
    print('[ERROR] Please run all previous notebooks in this section before you continue.')
    print('+++++++++++++++++++++++++++++++')

In [ ]:
print(iam_lambda_role_passed)

In [ ]:
if not iam_lambda_role_passed:
    print('+++++++++++++++++++++++++++++++')
    print('[ERROR] Please run all previous notebooks in this section before you continue.')
    print('+++++++++++++++++++++++++++++++')
else:
    print('[OK]')

In [ ]:
%store -r iam_role_lambda_arn

In [ ]:
try:
    iam_role_lambda_arn
except NameError:
    print('+++++++++++++++++++++++++++++++')
    print('[ERROR] Please run all previous notebooks in this section before you continue.')
    print('+++++++++++++++++++++++++++++++')

In [ ]:
print(iam_role_lambda_arn)

# Review Lambda Function Code 

In [ ]:
!pygmentize src/lambda_function.py

# Zip The Function Code

In [ ]:
!zip src/DeliverKinesisAnalyticsToCloudWatch.zip src/lambda_function.py

# Load the .zip File as Binary Code

In [ ]:
with open('src/DeliverKinesisAnalyticsToCloudWatch.zip', 'rb') as f: 
    code = f.read()

# Create The Lambda Function

In [ ]:
from botocore.exceptions import ClientError

try: 
    response = lam.create_function(
        FunctionName='{}'.format(lambda_fn_name),
        Runtime='python2.7',
        Role='{}'.format(iam_role_lambda_arn),
        Handler='src/lambda_function.lambda_handler',
        Code={
            'ZipFile': code
        },
        Description='Deliver output records from Kinesis Analytics application to CloudWatch.',
        Timeout=60,
        MemorySize=128,
        Publish=True
    )
    print('Lambda Function {} successfully created.'.format(lambda_fn_name))

except ClientError as e:
    if e.response['Error']['Code'] == 'ResourceConflictException':
        print('Lambda Function {} already exists. This is OK.'.format(lambda_fn_name))
    else:
        print('Error: {}'.format(e))

In [ ]:
response = lam.get_function(FunctionName=lambda_fn_name)
print(json.dumps(response, indent=4, sort_keys=True, default=str))

In [ ]:
response = lam.get_function(FunctionName=lambda_fn_name)

lambda_fn_arn = response['Configuration']['FunctionArn']
print(lambda_fn_arn)

In [ ]:
%store lambda_fn_arn

# Review Lambda Function

In [ ]:
from IPython.core.display import display, HTML
        
display(HTML('<b>Review <a target="blank" href="https://console.aws.amazon.com/lambda/home?region={}#/functions/{}"> Lambda Function</a></b>'.format(region, lambda_fn_name)))


# Store Variables for Next Notebooks

In [ ]:
%store

In [ ]:
%%javascript
Jupyter.notebook.save_checkpoint();
Jupyter.notebook.session.delete();